In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from dateutil import parser
import requests
import re
import dateparser
import pickle
from nltk.stem import SnowballStemmer
import nltk as nltk

nltk.download('stopwords')
stemmer=SnowballStemmer('portuguese')
stopwords = nltk.corpus.stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rhenanqueiroz/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Teste g1

In [ ]:
lista_noticias = []

response = requests.get('https://g1.globo.com/resumo-do-dia/noticia/2020/01/27/segunda-feira-27-de-janeiro.ghtml')

content = response.content

site = BeautifulSoup(content, 'html.parser')

# HTML da notícia
noticias = site.findAll('div', attrs={'class': 'feed-post-body'})

for noticia in noticias:
  # Título
  titulo = noticia.find('a', attrs={'class': 'feed-post-link'})

  # print(titulo.text)
  # print(titulo['href']) # link da notícia

  site_noticia = BeautifulSoup(requests.get(titulo['href']).content, 'html.parser')
  textos_noticia = site_noticia.findAll('p', class_="content-text__container")
  texto_noticia = " ".join([noti.getText() for noti in textos_noticia])

  # Subtítulo: div class="feed-post-body-resumo"
  subtitulo = noticia.find('div', attrs={'class': 'feed-post-body-resumo'})

  if (subtitulo):
    # print(subtitulo.text)
    lista_noticias.append([titulo.text, subtitulo.text, titulo['href'], texto_noticia])
  else:
    lista_noticias.append([titulo.text, '', titulo['href'], texto_noticia])


news = pd.DataFrame(lista_noticias, columns=['Título', 'Subtítulo', 'Link', 'Noticia'])
news
#news.to_excel('noticias.xlsx', index=False)

,Título,Subtítulo,Link,Noticia
0,STF nega pedido para obrigar Alcolumbre a marc...,Bolsonaro indicou ex-ministro para cadeira no ...,https://g1.globo.com/politica/noticia/2021/10/...,O ministro do Supremo Tribunal Federal (STF) ...
1,Moraes prorroga inquéritos sobre 'milícia digi...,Depoimento do presidente em uma das investigaç...,https://g1.globo.com/politica/noticia/2021/10/...,O ministro do Supremo Tribunal Federal (STF) ...
2,Justiça impede Sérgio Camargo de nomear ou exo...,Presidente da entidade foi denunciado por pers...,https://g1.globo.com/df/distrito-federal/notic...,"A 21ª Vara do Trabalho de Brasília decidiu, n..."
3,"7 presos, o alvo, disputa do tráfico: o que se...",Brasileiros foram detidos pela polícia paragua...,https://g1.globo.com/ms/mato-grosso-do-sul/not...,"Na madrugada de sábado (10), quatro pessoas f..."
4,VÍDEO: vítimas foram executadas logo após entr...,,https://g1.globo.com/ms/mato-grosso-do-sul/not...,Imagens de câmeras de segurança de uma loja m...
5,Ações da Embraer sobem 5% após venda de 100 ae...,"NetJets, de Warren Buffett, fez encomenda que ...",https://g1.globo.com/economia/noticia/2021/10/...,As ações da Embraer tiveram alta de 5% nesta ...
6,Dólar sobe mais e fecha no maior valor em quas...,,https://g1.globo.com/economia/noticia/2021/10/...,O dólar fechou em alta nesta segunda-feira (1...
7,Queda de avião de pequeno porte provoca incênd...,,https://g1.globo.com/mundo/noticia/2021/10/11/...,Um pequeno avião caiu em um bairro no subúrbi...
8,Itamaraty revoga nomeação de eliminado por cot...,"Lucas Siqueira foi eliminado por fraude, mas e...",https://g1.globo.com/df/distrito-federal/notic...,O Ministério das Relações Exteriores (MRE) re...
9,Imagens aéreas mostram destruição causada por ...,Fenômeno provocou tempestade com granizo e for...,https://g1.globo.com/sp/sao-carlos-regiao/noti...,Imagens aéreas feitas por drones mostraram o ...


In [ ]:
import datetime
weekday_name = ["segunda-feira", "terca-feira", "quarta-feira", "quinta-feira", "sexta-feira", "sabado", "domingo"]
months = ["Unknown", "janeiro", "fevereiro", "marco", "abril", "maio", "junho", "julho", "agosto", "setembro", "outubro","novembro", "dezembro"]

def return_url(dt):
    prefix = 'https://g1.globo.com/resumo-do-dia/noticia/'
    data_yyyy_mm_dd = str(dt.year) + '/' + dt.strftime('%m') + '/' + dt.strftime('%d') + '/'

    data_extenso = weekday_name[dt.weekday()] + '-' + str(dt.day) + '-de-' + months[dt.month] #para datas em que mostram 7-janeiro
    #data_extenso = weekday_name[dt.weekday()] + '-' + str(dt.strftime('%d')) + '-de-' + months[dt.month] #para datas em que mostram 07-janeiro
    sufix = '.ghtml'

    link_url = prefix + data_yyyy_mm_dd + data_extenso + sufix
    return link_url


In [ ]:
from dateutil import parser
import requests

headers = {
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.8',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
}

def df_noticias(url):
    '''
    Função para capturar o título, texto e data de publicação de cada noticia no site resumo do g1
    '''

    # transforma o objeto url na forma do bs4
    site = BeautifulSoup(requests.get(url, headers = headers).content, 'html.parser')

    #encontra todos os links de noticias do site de resumo diário
    links_noticias = site.findAll('p', attrs={'class': 'content-text__container'})
    lista_links = [link.find('a').get('href') for link in links_noticias if link.find('a') != None]

    #Cria lista vazia das noticias do dia com os links
    lista_noticias = []
    try:
        for link in lista_links:

            #Entra no link de cada noticia
            site_noticia = BeautifulSoup(requests.get(link, headers = headers).content, 'html.parser')

            #Pega o título
            titulo = site_noticia.find('h1', attrs={'class': 'content-head__title'})

            #Pega a data de publicação
            data = site_noticia.find('time', attrs={'itemprop': 'datePublished'})
            try:
                date_time_str = data.get('datetime')
                date_time = parser.parse(date_time_str).date()
            except:
                pass

            #Pega o conteúdo textual
            textos_noticia = site_noticia.findAll('p', class_="content-text__container")
            texto_noticia = " ".join([noti.getText() for noti in textos_noticia])

            if (titulo and data and texto_noticia):
                lista_noticias.append([link, titulo.text, date_time, texto_noticia])
            else:
                pass
    except:
        pass

    #retorna o dataframe com as infos
    return pd.DataFrame(lista_noticias, columns=['link','titulo', 'data_pub','noticias'])

In [ ]:
def salva_bases(data_in, data_out, ref):
    df = pd.DataFrame()
    lista_datas = pd.bdate_range(start=data_in, end=data_out)
    for data in lista_datas:
        url = return_url(data)
        df = df.append(df_noticias(url))
    df = df.reset_index().drop(columns = ['index'])

    caminho = "/Users/rhenanqueiroz/Documents/GitHub/fakenews/materias_g1/g1_" + ref + ".pkl"
    df.to_pickle(caminho)

In [ ]:
#salva_bases('08/01/2018', '08/31/2018', "201808")
#salva_bases('09/01/2018', '09/29/2018', "201809")
#salva_bases('10/01/2018', '10/29/2018', "201810")
#salva_bases('11/01/2018', '11/30/2018', "201811")
#salva_bases('12/01/2018', '12/31/2018', "201812")
#salva_bases('10/01/2018', '10/31/2018', "201810")
#salva_bases('01/01/2019', '01/31/2019', "201901")
#salva_bases('02/01/2019', '02/28/2019', "201902")
#salva_bases('03/01/2019', '03/31/2019', "201903")
#salva_bases('04/01/2019', '04/30/2019', "201904")
#salva_bases('05/01/2019', '05/31/2019', "201905")
#salva_bases('06/01/2019', '06/30/2019', "201906")
#salva_bases('07/01/2019', '07/31/2019', "201907")
#salva_bases('08/01/2019', '08/31/2019', "201908")
#salva_bases('09/01/2019', '09/30/2019', "201909")
#salva_bases('10/01/2019', '10/31/2019', "201910")
#salva_bases('11/01/2019', '11/30/2019', "201911")
#salva_bases('12/01/2019', '12/31/2019', "201912")
#salva_bases('01/01/2020', '01/31/2020', "202001")
#salva_bases('02/01/2020', '02/29/2020', "202002")
#salva_bases('03/01/2020', '03/31/2020', "202003")
#salva_bases('04/01/2020', '04/30/2020', "202004")
#salva_bases('05/01/2020', '05/31/2020', "202005")
#salva_bases('06/01/2020', '06/30/2020', "202006")
#salva_bases('07/01/2020', '07/30/2020', "202007")
#salva_bases('08/01/2020', '08/31/2020', "202008")
#salva_bases('09/01/2020', '09/30/2020', "202009")
#salva_bases('10/01/2020', '10/31/2020', "202010")
#salva_bases('11/01/2020', '11/30/2020', "202011")
#salva_bases('12/01/2020', '12/31/2020', "202012")
#salva_bases('01/01/2021', '01/31/2020', "202101")
#salva_bases('02/01/2021', '02/28/2021', "202102")
#salva_bases('03/01/2021', '03/31/2021', "202103")
#salva_bases('04/01/2021', '04/30/2021', "202104")
#salva_bases('05/01/2021', '05/31/2021', "202105")
#salva_bases('06/01/2021', '06/30/2021', "202106")
#salva_bases('07/01/2021', '07/30/2021', "202107")
#salva_bases('08/01/2021', '08/31/2021', "202108")
#salva_bases('09/01/2021', '09/30/2021', "202109")
#salva_bases('10/01/2021', '10/25/2021', "202110")

In [ ]:
import pickle
df_g1 = pd.DataFrame(columns=['link', 'titulo', 'data_pub', 'noticias']) # Creates an empty list

for root, dirs, files in os.walk("/Users/rhenanqueiroz/Documents/GitHub/fakenews/materias_g1/textos/"):
    for file in files:
        print(file)
        opencontacts = open(os.getcwd() + "/materias_g1/" + file, 'rb')
        loadedcontacts = pickle.load(opencontacts)
        df_g1 = df_g1.append(loadedcontacts)

df_g1


g1_201909.pkl
g1_201908.pkl
g1_202008.pkl
g1_202009.pkl
g1_201809.pkl
g1_201808.pkl
g1_202108.pkl
g1_202109.pkl
g1_202107.pkl
g1_202106.pkl
g1_202104.pkl
g1_202110.pkl
g1_202105.pkl
g1_202101.pkl
g1_201811.pkl
g1_201810.pkl
g1_202102.pkl
g1_201812.pkl
g1_202103.pkl
g1_202004.pkl
g1_202010.pkl
g1_202011.pkl
g1_202005.pkl
g1_201901.pkl
g1_201903.pkl
g1_202007.pkl
g1_202006.pkl
g1_202012.pkl
g1_201902.pkl
g1_201906.pkl
g1_201912.pkl
g1_202002.pkl
g1_202003.pkl
g1_201907.pkl
g1_201911.pkl
g1_201905.pkl
g1_202001.pkl
g1_201904.pkl
g1_201910.pkl


,link,titulo,data_pub,noticias
0,https://g1.globo.com/politica/noticia/2019/09/...,"'Um dos dois ou os dois perderão a cabeça', di...",2019-09-02,O presidente Jair Bolsonaro informou nesta se...
1,https://g1.globo.com/politica/noticia/2019/09/...,Bolsonaro diz que vai à ONU 'nem que seja de c...,2019-09-02,O presidente Jair Bolsonaro afirmou nesta seg...
2,https://g1.globo.com/natureza/noticia/2019/09/...,Focos de queimadas na Amazônia são quase 20 mi...,2019-09-02,Até o dia 29 de setembro foram registrados 19...
3,https://g1.globo.com/natureza/noticia/2019/09/...,Agosto tem o maior número de focos de queimada...,2019-09-01,As queimadas no bioma Amazônia aumentaram 196...
4,https://g1.globo.com/politica/noticia/2019/09/...,38% reprovam e 29% aprovam o governo Bolsonaro...,2019-09-02,Pesquisa Datafolha divulgada nesta segunda-fe...
...,...,...,...,...
353,https://g1.globo.com/mundo/noticia/2019/10/31/...,Câmara dos EUA aprova regras para seguir inqué...,2019-10-31,Os democratas na Câmara dos Deputados dos Est...
354,https://g1.globo.com/loterias/noticia/2019/10/...,Governo autoriza a Caixa a reajustar preço das...,2019-10-31,O Ministério da Economia autorizou a Caixa Ec...
355,https://globoesporte.globo.com/ginastica-artis...,"Acusado de abuso sexual, ex-técnico da ginásti...",2019-10-31,O ex-técnico da seleção masculina de ginástic...
356,https://g1.globo.com/pop-arte/noticia/2019/10/...,Halloween: veja fantasias dos famosos em festa...,2019-10-31,O Halloween é comemorado nesta quinta-feira (...


In [ ]:
df = df_g1.reset_index().drop(columns = ['index'])

caminho = "/Users/rhenanqueiroz/Documents/GitHub/fakenews/g1_geral.pkl"
df.to_pickle(caminho)

# Terça livre

In [ ]:
url = 'https://tercalivre.com.br/mais-noticias/page/800/'
# transforma o objeto url na forma do bs4
site = BeautifulSoup(requests.get(url).content, 'html.parser')

#encontra todos os links de noticias do site de resumo diário
links_noticias = site.findAll('h2', attrs={'class': 'entry-title h7'})
lista_links = [link.find('a').get('href') for link in links_noticias if link.find('a') != None]


In [ ]:
lista_links[0]

'https://tercalivre.com.br/filho-de-renan-calheiros-enfrentara-o-psdb-em-alagoas-o-senador-vai-disputar-com-os-ministros-de-temer/'

In [ ]:
def df_noticias_terca_livre(url):
    '''
    Função para capturar o título, texto e data de publicação de cada noticia no site resumo do terça livre
    '''
    import re
    import dateparser
    # transforma o objeto url na forma do bs4
    site = BeautifulSoup(requests.get(url).content, 'html.parser')

    #encontra todos os links de noticias do site de resumo diário
    links_noticias = site.findAll('h2', attrs={'class': 'entry-title h7'})
    lista_links = [link.find('a').get('href') for link in links_noticias if link.find('a') != None]

    #Cria lista vazia das noticias do dia com os links
    lista_noticias = []

    for link in lista_links:

        #Entra no link de cada noticia
        site_noticia = BeautifulSoup(requests.get(link).content, 'html.parser')

        #Pega o título
        titulo = site_noticia.find('h1', attrs={'class': 'entry-title h1'})

        #Pega a data de publicação

        data = site_noticia.find('span', attrs={'class': 'updated'})
        try:
            date_time_str = dateparser.parse(data.get_text(), languages=["pt"]).strftime('%Y-%m-%d')
            date_time = parser.parse(date_time_str).date()
        except:
            pass

        #Pega o conteúdo textual
        textos_noticia = site_noticia.findAll('p')
        texto_noticia = " ".join([noti.getText() for noti in textos_noticia])
        texto_noticia2 = re.sub(r"Comentário Nome [\s\S]*/+", ' ', texto_noticia)

        if (titulo and data and texto_noticia2 and "#" not in titulo.text and len(texto_noticia2)>100):
            lista_noticias.append([link, titulo.text, date_time, texto_noticia2])
        else:
            pass

    #retorna o dataframe com as infos
    return pd.DataFrame(lista_noticias, columns=['link','titulo', 'data_pub','noticias'])

In [ ]:
df = df_noticias_terca_livre('https://tercalivre.com.br/mais-noticias/page/800/')
df

,link,titulo,data_pub,noticias
0,https://tercalivre.com.br/intervencao-federal-...,Intervenção Federal: uma guerra onde só um lad...,2018-02-21,"Ainda ontem, 20, um sargento do exército morre..."
1,https://tercalivre.com.br/diante-do-que-o-rj-e...,"Diante do que o RJ enfrenta, a preocupação da ...",2018-02-20,A Rede Globo de televisão flagrou um homem da ...
2,https://tercalivre.com.br/analista-politico-qu...,Analista político que acertou resultados de Tr...,2018-02-20,"Em entrevista ao InfoMoney, o analista polític..."
3,https://tercalivre.com.br/prazo-para-defesa-de...,Prazo para defesa de Lula encerra hoje. TRF-4 ...,2018-02-20,O prazo para que a defesa do ex-presidente con...
4,https://tercalivre.com.br/grave-paraclitus-afi...,GRAVE: Paraclitus afirma que Campanha da Frate...,2018-02-20,Uma denúncia gravíssima foi publicada pelo sit...
5,https://tercalivre.com.br/depois-da-intervenca...,"Depois da intervenção, ministro já fala em can...",2018-02-20,"Em entrevista à Rádio Gaúcha, o ministro da Ca..."
6,https://tercalivre.com.br/filho-de-renan-calhe...,Filho de Renan Calheiros enfrentará o PSDB em ...,2018-02-20,"Em Alagoas, tanto o senador Renan Calheiros (M..."
7,https://tercalivre.com.br/maduro-desafia-trump...,Maduro desafia Trump em nome da revolução boli...,2018-02-20,O ditador da Venezuela Nicolás Maduro (que alg...
8,https://tercalivre.com.br/juiz-reage-assalto-p...,"Juiz reage a assalto, persegue criminosos e fe...",2018-02-19,Uma tentativa de assalto à casa de um juiz na ...
9,https://tercalivre.com.br/homens-que-tatuaram-...,Homens que tatuaram testa de adolescente são c...,2018-02-19,Os dois homens que tatuaram a testa de um adol...


In [ ]:
def salva_bases_tl(cent1,cent2):
    df = pd.DataFrame()

    for numero in range(cent1,cent2):
        url = 'https://tercalivre.com.br/mais-noticias/page/' + str(numero) + '/'
        df = df.append(df_noticias_terca_livre(url))
    df = df.reset_index().drop(columns = ['index'])

    caminho = "/Users/rhenanqueiroz/Documents/GitHub/fakenews/tl_" + str(cent2) + ".pkl"
    df.to_pickle(caminho)

In [ ]:
salva_bases_tl(100,101)

In [ ]:
df_2017 = pd.read_csv('/Users/rhenanqueiroz/Downloads/articles_pre_processed.csv', sep = '|')
df_2017

,Unnamed: 0,Text
0,0,com a possibilidade de uma condenacao impedir ...
1,1,para oumou sangare cantora e ativista malinesa...
2,2,tres reportagens da folha foram vencedoras do ...
3,3,a disney divulgou na noite desta segundafeira ...
4,4,o cbss banco da holding elopar dos socios brad...
...,...,...
165661,165661,o tucano beto richa tinha tudo para comecar se...
165662,165662,o economista renan filho pmdb assume nesta qui...
165663,165663,destaques da programacao desta quintafeira 1º ...
165664,165664,o lider nortecoreano kim jongun disse nesta qu...


# Senso Incomum

In [ ]:
import datetime
def return_url_senso_incomum(dt):
    prefix = 'https://sensoincomum.org/'
    data_yyyy_mm_dd = str(dt.year) + '/' + dt.strftime('%m') + '/' + dt.strftime('%d') + '/'

    link_url = prefix + data_yyyy_mm_dd
    return link_url

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
def df_noticias_senso_incomum(url):
    '''
    Função para capturar o título, texto e data de publicação de cada noticia no site resumo do terça livre
    '''
    import re
    import dateparser
    # transforma o objeto url na forma do bs4
    site = BeautifulSoup(requests.get(url).content, 'html.parser')

    #encontra todos os links de noticias do site de resumo diário
    links_noticias = site.findAll('a', attrs={'class': 'tt-post-title titulo-cat'})
    lista_links = [link.get('href') for link in links_noticias]
    if lista_links:
        #Cria lista vazia das noticias do dia com os links
        lista_noticias = []

        for link in lista_links:

            #Entra no link de cada noticia
            site_noticia = BeautifulSoup(requests.get(link).content, 'html.parser')



            #Pega o título
            titulo = site_noticia.find('h1', attrs={'class': 'c-h1 text-center'})


            #Pega a data de publicação
            data = site_noticia.find('span', attrs={'class': 'tt-post-date-single'})
            try:
                date_time_str = dateparser.parse(data.get_text(), languages=["pt"]).strftime('%Y-%m-%d')
                date_time = parser.parse(date_time_str).date()
            except:
                pass

            #Pega o conteúdo textual
            container = site_noticia.find('article')
            textos_noticia = container.findAll('p')
            texto_noticia = " ".join([noti.getText() for noti in textos_noticia])

            if (titulo and data and texto_noticia and len(texto_noticia)>100):
                lista_noticias.append([link, titulo.text, date_time, texto_noticia])
            else:
                pass


        #retorna o dataframe com as infos
        return pd.DataFrame(lista_noticias, columns=['link','titulo', 'data_pub','noticias'])

In [ ]:
def salva_bases_senso_incomum(data_in, data_out, ref):
    df = pd.DataFrame()
    lista_datas = pd.bdate_range(start=data_in, end=data_out)
    for data in lista_datas:
        url = return_url(data)
        df = df.append(df_noticias(url))
    df = df.reset_index().drop(columns = ['index'])

    caminho = "/Users/rhenanqueiroz/Documents/GitHub/fakenews/senso_incomum_" + ref + ".pkl"
    df.to_pickle(caminho)

In [ ]:
#salva_bases_senso_incomum('01-01-2017', '10-28-2021',"geral")

# Jornal da cidade online

In [ ]:
def return_url_jornal_cidade_online(ref):
    prefix = 'https://www.jornaldacidadeonline.com.br/noticias/'
    sufix = str(ref) + '/'
    link_url = prefix + sufix
    return link_url

In [ ]:
def df_jornal_cidade_online(url):
    '''
    Função para capturar o título, texto e data de publicação de cada noticia no site resumo do terça livre
    '''
    import re
    import dateparser
    lista_noticias = []
    try:
        #Entra no link de cada noticia
        site_noticia = BeautifulSoup(requests.get(url).content, 'html.parser')

        #Pega o título
        titulo = site_noticia.find('h1', attrs={'class': 'post__title'})

        #Pega a data de publicação
        data = site_noticia.find('small', attrs={'class': 'post__date'})
        try:
            date_time_str = dateparser.parse(data.get_text(), languages=["pt"]).strftime('%Y-%m-%d')
            date_time = parser.parse(date_time_str).date()
        except:
            pass

        #Pega o conteúdo textual
        container = site_noticia.find('div', attrs={'class': 'post__description'})
        textos_noticia = container.findAll('p')
        texto_noticia = " ".join([noti.getText() for noti in textos_noticia])
        texto_noticia = re.sub(r'Em tempos de "censura", precisamos da ajuda do nosso leitor. [\s\S]*/+', ' ', texto_noticia)
        texto_noticia = re.sub(r"ATENÇÃO! Agora você tem a opção de assinatura do JCO com boleto! [\s\S]*/+", ' ', texto_noticia)
        texto_noticia = re.sub(r"Faça a sua assinatura. [\s\S]*/+", ' ', texto_noticia)
        texto_noticia = re.sub(r"Um movimento anônimo, criminoso e fascista está tentando destruir o Jornal da Cidade Online. [\s\S]*/+", ' ', texto_noticia)
        texto_noticia = re.sub(r"Precisamos da ajuda de todos os patriotas. [\s\S]*/+", ' ', texto_noticia)
        texto_noticia = re.sub(r"Fortaleça o jornalismo independente do Jornal da Cidade Online. [\s\S]*/+", ' ', texto_noticia)
        texto_noticia = re.sub(r"Ajude o JCO a continuar sobrevivendo com independência. [\s\S]*/+", ' ', texto_noticia)
        texto_noticia = re.sub(r"Você se incomoda com as publicidades no site\? [\s\S]*/+", ' ', texto_noticia)

        if (titulo and data and texto_noticia and len(texto_noticia)>100):
            lista_noticias.append([url, titulo.text, date_time, texto_noticia])
        else:
            pass

        #retorna o dataframe com as infos
        return pd.DataFrame(lista_noticias, columns=['link','titulo', 'data_pub','noticias'])
    except:
        pass

In [ ]:
def salva_bases_jornal_cidade_online(ref_in, ref_out, ref):
    df = pd.DataFrame()
    lista_datas = [i for i in range(ref_in, ref_out)]
    for data in lista_datas:
        url = return_url_jornal_cidade_online(data)
        df = df.append(df_jornal_cidade_online(url))
    df = df.reset_index().drop(columns = ['index'])

    caminho = "/Users/rhenanqueiroz/Documents/GitHub/fakenews/jornal_cidade_online_" + ref + ".pkl"
    df.to_pickle(caminho)

In [ ]:
# salva_bases_jornal_cidade_online(4700,34000,'geral')

# Brasil de fato

In [ ]:
def return_url_brasil_de_fato(ref):
    prefix = 'https://www.brasildefato.com.br/ultimas-noticias?pagina='
    sufix = str(ref) + '/'
    link_url = prefix + sufix
    return link_url

In [ ]:
def df_noticias_brasil_de_fato(url):
    '''
    Função para capturar o título, texto e data de publicação de cada noticia no site resumo do brasil de fato
    '''

    # transforma o objeto url na forma do bs4
    site = BeautifulSoup(requests.get(url).content, 'html.parser')

    #encontra todos os links de noticias do site de resumo diário
    links_noticias = site.findAll('a', attrs={'class': 'news-item'})
    lista_links = ['https://www.brasildefato.com.br' + link.get('href') for link in links_noticias]
    if lista_links:
        #Cria lista vazia das noticias do dia com os links
        lista_noticias = []

        for link in lista_links:

            try:

                #Entra no link de cada noticia
                site_noticia = BeautifulSoup(requests.get(link).content, 'html.parser')



                #Pega o título
                titulo = site_noticia.find('h1', attrs={'class': 'title'})


                #Pega a data de publicação
                data = site_noticia.find('time', attrs={'class': 'date'})
                try:
                    date_time_str = dateparser.parse(data.get_text(), languages=["pt"]).strftime('%Y-%m-%d')
                    date_time = parser.parse(date_time_str).date()
                except:
                    pass

                #Pega o conteúdo textual
                container = site_noticia.find('div', attrs={'class': 'content'})
                textos_noticia = container.findAll('p')
                texto_noticia = " ".join([noti.getText() for noti in textos_noticia])

                if (titulo and date_time and texto_noticia and len(texto_noticia)>100):
                    lista_noticias.append([link, titulo.text, date_time, texto_noticia])
                else:
                    pass
            except:
                pass


        #retorna o dataframe com as infos
        return pd.DataFrame(lista_noticias, columns=['link','titulo', 'data_pub','noticias'])

In [ ]:
def salva_bases_brasil_de_fato(ref_in, ref_out, ref):
    df = pd.DataFrame()
    lista_datas = [i for i in range(ref_in, ref_out)]
    df = df.append([df_noticias_brasil_de_fato(return_url_brasil_de_fato(data)) for data in lista_datas])
    df = df.reset_index().drop(columns = ['index'])

    caminho = "/Users/rhenanqueiroz/Documents/GitHub/fakenews/brasil_de_fato_" + ref + ".pkl"
    df.to_pickle(caminho)

In [ ]:
#salva_bases_brasil_de_fato(1,1275,'geral')

# El pais

In [ ]:
def return_url_elpais(dt):
    prefix = 'https://brasil.elpais.com/acervo/'
    data_yyyy_mm_dd = str(dt.year) + '-' + dt.strftime('%m') + '-' + dt.strftime('%d') + '/'
    link_url = prefix + data_yyyy_mm_dd
    return link_url

In [ ]:
def df_noticias_elpais(url):
    '''
    Função para capturar o título, texto e data de publicação de cada noticia no site resumo do el pais
    '''

    # transforma o objeto url na forma do bs4
    site = BeautifulSoup(requests.get(url).content, 'html.parser')

    #encontra todos os links de noticias do site de resumo diário
    links_noticias_cont = site.findAll('h2', class_ = 'c_t')
    lista_links = ['https://brasil.elpais.com' + link.find('a').get('href') for link in links_noticias_cont]
    if lista_links:
        #Cria lista vazia das noticias do dia com os links
        lista_noticias = []

        for link in lista_links:

            try:

                #Entra no link de cada noticia
                site_noticia = BeautifulSoup(requests.get(link).content, 'html.parser')



                #Pega o título
                titulo = site_noticia.find('h1', attrs={'class': 'a_t'})


                #Pega a data de publicação
                data = site_noticia.find('a', attrs={'id': 'article_date_p'})
                try:
                    date_time_str = dateparser.parse(data.get_text(), languages=["pt"]).strftime('%Y-%m-%d')
                    date_time = parser.parse(date_time_str).date()
                except:
                    pass

                #Pega o conteúdo textual
                container = site_noticia.find('div', attrs={'class': 'a_c clearfix'})
                textos_noticia = container.findAll('p')
                texto_noticia = " ".join([noti.getText() for noti in textos_noticia])

                if (titulo and date_time and texto_noticia and len(texto_noticia)>100):
                    lista_noticias.append([link, titulo.text, date_time, texto_noticia])
                else:
                    pass
            except:
                pass


        #retorna o dataframe com as infos
        return pd.DataFrame(lista_noticias, columns=['link','titulo', 'data_pub','noticias'])

In [ ]:
def salva_bases_elpais(ref_in, ref_out, ref):
    df = pd.DataFrame()
    lista_datas = pd.date_range(start=ref_in, end=ref_out)
    for data in lista_datas:
        url = return_url_elpais(data)
        df = df.append(df_noticias_elpais(url))
    df = df.reset_index().drop(columns = ['index'])
    caminho = "/Users/rhenanqueiroz/Documents/GitHub/fakenews/elpais_" + ref + ".pkl"
    df.to_pickle(caminho)

In [ ]:
#salva_bases_elpais('2017-01-01','2021-10-30','geral')

# Conexão política

In [ ]:
def return_url_conexao_politica(ref):
    prefix = 'https://www.conexaopolitica.com.br/todos-conteudos/page/'
    sufix = str(ref) + '/'
    link_url = prefix + sufix
    return link_url

In [ ]:
def df_noticias_conexao_politica(url):
    '''
    Função para capturar o título, texto e data de publicação de cada noticia no site resumo do conexao politica
    '''

    # transforma o objeto url na forma do bs4
    site = BeautifulSoup(requests.get(url).content, 'html.parser')

    #encontra todos os links de noticias do site de resumo diário
    links_noticias_cont = site.findAll('div', class_ = 'zox-art-title')
    lista_links = [link.find('a').get('href') for link in links_noticias_cont]
    if lista_links:
        #Cria lista vazia das noticias do dia com os links
        lista_noticias = []

        for link in lista_links:

            try:

                #Entra no link de cada noticia
                site_noticia = BeautifulSoup(requests.get(link).content, 'html.parser')



                #Pega o título
                titulo = site_noticia.find('h1')


                #Pega a data de publicação
                data = site_noticia.find('time').get('datetime')
                date_time = parser.parse(data).date()

                #Pega o conteúdo textual
                container = site_noticia.find('div', attrs={'class': 'zox-post-body left zoxrel zox100'})
                textos_noticia = container.findAll('p')
                texto_noticia = " ".join([noti.getText() for noti in textos_noticia])

                if (titulo and date_time and texto_noticia and len(texto_noticia)>100):
                    lista_noticias.append([link, titulo.text, date_time, texto_noticia])
                else:
                    pass
            except:
                pass


        #retorna o dataframe com as infos
        return pd.DataFrame(lista_noticias, columns=['link','titulo', 'data_pub','noticias'])

In [ ]:
def salva_bases_conexao_politica(ref_in, ref_out, ref):
    df = pd.DataFrame()
    lista_datas = [i for i in range(ref_in, ref_out)]
    df = df.append([df_noticias_conexao_politica(return_url_conexao_politica(data)) for data in lista_datas])
    df = df.reset_index().drop(columns = ['index'])

    caminho = "/Users/rhenanqueiroz/Documents/GitHub/fakenews/conexao_politica_" + ref + ".pkl"
    df.to_pickle(caminho)

In [ ]:
#salva_bases_conexao_politica(1, 1592, 'geral')

# Agregando todos os textos

In [ ]:
df_g1 = pd.read_pickle('g1_geral.pkl')
df_elpais = pd.read_pickle('elpais_geral.pkl')
df_jco = pd.read_pickle('jornal_cidade_online_geral.pkl')
df_si = pd.read_pickle('senso_incomum_geral.pkl')
df_bdf = pd.read_pickle('brasil_de_fato_geral.pkl')
df_cnxpol = pd.read_pickle('conexao_politica_geral.pkl')

In [ ]:
df_geral = df_g1.append(df_elpais).append(df_jco).append(df_si).append(df_bdf).append(df_cnxpol)
len(df_geral)

83256

# Limpeza dos dados

In [ ]:
import re
def limpa_texto(texto):
    # remove \n e \r
    c = re.sub(r'\n', ' ', texto)
    c = re.sub(r'\r', ' ', c)

    # remove emoction
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)

    c = emoji_pattern.sub(r'', c)

    # remove os @ listados
    c = re.sub(r'@[\w./-]+', '', c)

    # remove os sites
    c = re.sub(r'http[\w./:-]+', '', c)

    # remove tag html
    c = re.sub(r'<.*?>','', c)

    # remove caracteres não alfabéticos
    c = c.replace(':', ' ')
    c = c.replace(';', ' ')
    c = c.replace('.', ' ')
    c = c.replace('/', ' ')
    c = re.sub(r'R\$', '', c)
    c = re.sub(r'\W', ' ', c)
    c = re.sub('[‘’“”…]', '', c)
    c = re.sub('\w*\d\w*', '', c)

    # remove espaços adicionais
    c = re.sub(r'\s+', ' ', c)

    # remove espacos adicionais no inicio das frases
    c = re.sub(r'^\s+', '', c)

    # remove espacos adicionais no final das frases
    c = re.sub(r'\s+$', '', c)

    #remove palavras que tenham mais de 3 letras repetidas seguidas e qq outro
    c = re.sub(r'\w+?(\w)\1{2,}(\w)+\b', '', c)

    #remove risadas
    c = re.sub(r'(h?a*ha+h[ha]*.)', '', c)

    #retira acentos
    c = c.replace("á","a")
    c = c.replace("ã","a")
    c = c.replace("õ","o")
    c = c.replace("é","e")
    c = c.replace("í","i")
    c = c.replace("ó","o")
    c = c.replace("ô","o")
    c = c.replace("ú","u")
    c = c.replace("ç","c")

    return c

def stemmatiza(texto):
    doc = nltk.word_tokenize(texto)
    return ' '.join(stemmer.stem(token.lower()) for token in doc)

def remove_stop(texto):
    doc = nltk.word_tokenize(texto)
    return " ".join(token for token in doc if token not in stopwords)


In [ ]:
df_geral['noticias_limpas'] = df_geral['noticias'].apply(lambda x: limpa_texto(x))\
                                                  .apply(lambda x: remove_stop(x))

df_geral

In [ ]:
df_geral.to_pickle('todas_noticias.pkl')